In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('datos/bikes.csv', index_col = 0)

In [4]:
df.sample(10)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
716,717,18-12-2019,winter,1,12,0,2,1,1,16.844153,20.48540,66.6250,14.834068,433,5124,5557
476,477,22-04-2019,summer,1,4,0,0,0,3,16.263347,19.47520,83.5417,23.084582,120,907,1027
104,105,15-04-2018,summer,0,4,1,5,0,1,18.313347,22.09565,67.1250,15.167125,642,2484,3126
264,265,22-09-2018,autumn,0,9,0,4,1,2,25.761653,27.74815,90.2083,8.584375,555,4240,4795
336,337,03-12-2018,winter,0,12,0,6,0,1,12.265847,15.53020,61.2917,6.417481,706,2908,3614
495,496,11-05-2019,summer,1,5,0,5,1,1,21.866653,26.04165,36.0417,15.874779,1319,5711,7030
251,252,09-09-2018,autumn,0,9,0,5,1,2,26.650000,28.94730,89.7917,8.333393,417,3127,3544
703,704,05-12-2019,winter,1,12,0,3,1,1,17.971653,21.40060,48.5000,21.709407,331,5398,5729
283,284,11-10-2018,winter,0,10,0,2,1,2,23.233347,27.30480,80.8750,9.583814,667,3896,4563
330,331,27-11-2018,winter,0,11,0,0,0,1,18.825847,22.79000,69.8333,13.999918,810,2261,3071


day.csv have the following fields:
	
	- instant: record index
	- dteday : date
	- season : season (spring, summer, autumn, winter)
	- yr : year (0: 2018, 1:2019)
	- mnth : month ( 1 to 12)
	- holiday : weather day is a holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
	- weekday : day of the week
	- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
	+ weathersit : 
		- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
		- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
		- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
		- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
	- temp : temperature in Celsius
	- atemp: feeling temperature in Celsius
	- hum: humidity
	- windspeed: wind speed
	- casual: count of casual users
	- registered: count of registered users
	- cnt: count of total rental bikes including both casual and registered